# hyperparameterTunningResults5
Andrew E. Davidson  
aedavids@ucsc.edu  
4/23/2024

see extraCellularRNA/deconvolutionAnalysis/doc/bestCurratedNotes.md for a discussion about tunning 

**<span style="color:red;background-color:yellow">AEDWIP</span>**  


In [1]:
import ipynbname

# use display() to print an html version of a data frame
# useful if dataFrame output is not generated by last like of cell
from IPython.display import display

import numpy as np
import pandas as pd
# display all columns
pd.set_option('display.max_columns', None)

import pathlib as pl
import pprint as pp
import os
import sys

/data/tmp/ipykernel_1261507/2134628107.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# setting the python path allows us to run python scripts from using
# the CLI. 
ORIG_PYTHONPATH = os.environ['PYTHONPATH']

notebookPath = ipynbname.path()
deconvolutionModules = notebookPath.parent.joinpath("../../python")
print("deconvolutionModules: {}\n".format(deconvolutionModules))

PYTHONPATH = ORIG_PYTHONPATH + f':{deconvolutionModules}'
print("PYTHONPATH: {}\n".format(PYTHONPATH))

os.environ["PYTHONPATH"] = PYTHONPATH
PYTHONPATH = os.environ["PYTHONPATH"]
print("PYTHONPATH: {}\n".format(PYTHONPATH))

# to be able to import our local python files we need to set the sys.path
# https://stackoverflow.com/a/50155834
sys.path.append( str(deconvolutionModules) )
print("\nsys.path:\n{}\n".format(sys.path))

deconvolutionModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python


sys.path:
['/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning', '/private/home/aedavids/extraCellularRNA/src', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python311.zip', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/lib-dynload', '', '/private/h

In [3]:
from analysis.hyperParameterTunningMetrics import metricsRunner, elifeCols, lungCols
from analysis.hyperParameterTunningMetrics import findSummaryMetricsCols
from analysis.utilities import findAllCategories, findAllGenes
from analysis.utilities import findIntersectionsWithDegree
from analysis.utilities import loadDictionary

In [4]:
root = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category"
notebookName = ipynbname.name()
outDir = f'{root}/hyperParameter/{notebookName}.out'
print( f'output dir: \n{outDir}' )
os.makedirs(outDir, exist_ok=True)

output dir: 
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults5.out


In [5]:
def evaluate(
    root: str,
    outDir: str,    
    resultsDirs : list[str],
    outFilePrefix : str,
    metric : str,
    stageName : str,
    threshold : float,
    verbose : bool = False,
    ) -> tuple[pd.DataFrame, pd.DataFrame] :
    '''
    returns:
        df : aedwip

        bellowThresholdDF:
            example:
                    stage	        category	               value
                id			
                0	best10GTEx_TCGA	ACC	                        True
                2	best10GTEx_TCGA	Adipose_Visceral_Omentum	True
                5	best10GTEx_TCGA	Artery_Coronary	            True
                7	best10GTEx_TCGA	BLCA	                    True
                8	best10GTEx_TCGA	BRCA	                    True
            
    '''

    retDF, retBellowThresholdDF = metricsRunner(root, outDir, outFilePrefix, resultsDirs, 
                           metric=metric, threshold=threshold, verbose=verbose)

    display( retDF.loc[:, findSummaryMetricsCols(metric) + elifeCols  ] )

    print(f'\n {stageName} classes < {threshold} {metric}')
    selectRowsBellow = retBellowThresholdDF.loc[:, "stage"] == stageName
    
    display( retBellowThresholdDF.loc[selectRowsBellow, 'category'] )

    return (retDF, retBellowThresholdDF)

In [6]:
def evaluateBest10LCFResults(
        threshold : float = 0.7,
        metric : str = 'sensitivity',
        verbose : bool = False):
    bestBes10LCFResultsDirs = [    
        "best10GTEx_TCGA",
        "best10LFC_GTEx_TCGA", 
        "best10CuratedDegree1",
        "best10CuratedDegree1_ce467ff",
        "best10LFC_CuratedDegree1",
    ]
    
    print(f'metric : {metric} threshold: {threshold}')
    stageName =  "best20FindAllDegree1_wl5" # stageName is part of the outputfile prefix
    best10LFCDF, bestLFCBellowThresholdDF = evaluate(
                                            root=root,
                                            outDir=outDir,
                                            resultsDirs=bestBes10LCFResultsDirs, 
                                            outFilePrefix=stageName,
                                            metric=metric,
                                            stageName=stageName,
                                            threshold=threshold,
                                            verbose=verbose,
                                            )
    
    return(best10LFCDF, bestLFCBellowThresholdDF)

evalBest10LFCSpecificityDF, evalBestLFCSpecificityBellowThresholdDF = \
    evaluateBest10LCFResults(threshold=0.96, metric='specificity')

# if we do not bind the results, jupyter lab will print them.
evalBest10LFCSensitivityDF, evalBestLFCSensitivityBellowThresholdDF = \
    evaluateBest10LCFResults(threshold=0.70, metric='sensitivity', verbose=False)

metric : specificity threshold: 0.96

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults5.out/best20FindAllDegree1_wl5.specificity.0.96.csv

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults5.out/best20FindAllDegree1_wl5.specificity.bellow.0.96.csv


id,mean_specificity,std_specificity,median_specificity,numGenes,numTypes,numDegree1,numAboveThreshold,LUAD,LUSC,COAD,READ,ESCA,LIHC,STAD,Whole_Blood
best10GTEx_TCGA,0.996361,0.004210,0.997,136,83,30,83,0.999,0.996,0.995,0.994,0.999,1.000,0.996,1.000
best10LFC_GTEx_TCGA,0.996229,0.006842,0.999,702,83,83,82,0.999,0.992,0.994,0.997,1.000,0.999,0.998,0.999
best10CuratedDegree1,0.997542,0.003030,0.998,716,83,83,83,1.000,0.994,0.993,0.991,0.998,0.999,1.000,1.000
best10CuratedDegree1_ce467ff,0.997855,0.002922,0.999,716,83,83,83,0.998,0.995,0.995,0.996,0.999,1.000,0.999,0.999
best10LFC_CuratedDegree1,0.997181,0.003745,0.998,757,83,83,83,1.000,0.997,0.987,0.989,0.999,1.000,1.000,1.000



 best20FindAllDegree1_wl5 classes < 0.96 specificity


Series([], Name: category, dtype: object)

metric : sensitivity threshold: 0.7

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults5.out/best20FindAllDegree1_wl5.sensitivity.0.7.csv

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults5.out/best20FindAllDegree1_wl5.sensitivity.bellow.0.7.csv


id,mean_sensitivity,std_sensitivity,median_sensitivity,numGenes,numTypes,numDegree1,numAboveThreshold,LUAD,LUSC,COAD,READ,ESCA,LIHC,STAD,Whole_Blood
best10GTEx_TCGA,0.675398,0.230238,0.703,136,83,30,42,0.453,0.455,0.500,0.536,0.270,0.789,0.347,0.987
best10LFC_GTEx_TCGA,0.673639,0.235245,0.701,702,83,83,42,0.379,0.462,0.684,0.482,0.027,0.659,0.209,0.996
best10CuratedDegree1,0.786482,0.206089,0.833,716,83,83,62,0.485,0.595,0.627,0.679,0.396,0.906,0.271,0.987
best10CuratedDegree1_ce467ff,0.808831,0.202235,0.871,716,83,83,63,0.841,0.691,0.652,0.607,0.369,0.874,0.409,1.000
best10LFC_CuratedDegree1,0.767373,0.201111,0.793,757,83,83,56,0.595,0.645,0.620,0.679,0.342,0.740,0.307,0.976



 best20FindAllDegree1_wl5 classes < 0.7 sensitivity


Series([], Name: category, dtype: object)

In [7]:
# 3/29 with Daniel
runOfInsterest = ["best10CuratedDegree1_ce467ff", "best10CuratedDegree1", ]
evalBest10LFCSpecificityDF.loc[ runOfInsterest ]

id,ACC,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,BLCA,BRCA,Bladder,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,CESC,CHOL,COAD,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Cervix_Endocervix,Colon_Sigmoid,Colon_Transverse,DLBC,ESCA,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,GBM,HNSC,Heart_Atrial_Appendage,Heart_Left_Ventricle,KICH,KIRC,KIRP,Kidney_Cortex,LGG,LIHC,LUAD,LUSC,Liver,Lung,MESO,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,OV,Ovary,PAAD,PCPG,PRAD,Pancreas,Pituitary,Prostate,READ,SARC,SKCM,STAD,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,TGCT,THCA,THYM,Testis,Thyroid,UCEC,UCS,UVM,Uterus,Vagina,Whole_Blood,mean_specificity,std_specificity,median_specificity,numGenes,numTypes,numDegree1,numAboveThreshold
best10CuratedDegree1_ce467ff,1.0,0.987,0.995,1.0,0.998,0.997,0.998,0.998,1.0,0.999,0.999,0.998,0.998,0.999,0.999,0.997,0.994,0.999,0.999,0.995,0.999,0.990,0.998,0.996,0.995,0.998,0.995,0.997,1.0,0.999,0.987,0.999,0.999,0.999,0.994,0.996,0.995,0.997,0.994,1.0,1.000,0.999,1.0,0.999,1.000,1.000,1.000,0.998,0.995,0.999,0.998,1.0,1.0,1.0,1.0,1.0,1.000,0.991,1.0,1.0,1.000,1.0,0.999,0.996,1.0,1.0,0.999,0.990,0.994,1.0,0.999,0.999,1.0,1.0,1.0,1.0,1.000,0.998,0.995,1.000,1.000,0.999,0.999,0.997855,0.002922,0.999,716,83,83,83
best10CuratedDegree1,1.0,0.989,0.996,1.0,0.998,0.998,0.999,0.994,1.0,0.998,0.996,0.998,0.998,0.999,0.999,0.998,0.994,1.000,0.997,0.995,0.999,0.987,0.999,0.998,0.995,0.999,0.993,0.998,1.0,0.998,0.986,0.998,0.997,0.998,0.994,0.997,0.996,0.998,0.995,1.0,0.997,0.999,1.0,0.998,0.999,0.999,0.999,1.000,0.994,0.999,0.997,1.0,1.0,1.0,1.0,1.0,0.999,1.000,1.0,1.0,0.999,1.0,0.997,0.991,1.0,1.0,1.000,0.989,0.995,1.0,0.995,1.000,1.0,1.0,1.0,1.0,0.996,0.996,0.996,0.998,0.998,0.995,1.000,0.997542,0.003030,0.998,716,83,83,83


## which classes in best10LFC_GTEx_TCGA under performed?
<span style="color:red;background-color:yellow">aedwip: look at output of foo() looks like we could manually add genes from best10CuratedDegree1_ce467ff" </span>


In [8]:
def foo(
    df : pd.DataFrame,
    stage : str = "best10LFC_GTEx_TCGA",
    ) -> pd.Series:
    '''
    TODO
    '''
    
    selectRows = df.loc[:,'stage'] == stage
    retSeries = df.loc[selectRows, "category"]
    return retSeries



stage = "best10LFC_GTEx_TCGA"
belowSeries = foo(evalBestLFCSensitivityBellowThresholdDF, stage)
print(f'belowSeries.shape : {belowSeries.shape}')

print('\n\n^^^^^^^^^^^^^^^')
display( evalBest10LFCSensitivityDF.loc[:, belowSeries.values] )

belowSeries.shape : (41,)


^^^^^^^^^^^^^^^


id,ACC,Adipose_Visceral_Omentum,Artery_Coronary,BLCA,BRCA,Bladder,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Breast_Mammary_Tissue,CESC,COAD,Colon_Sigmoid,Colon_Transverse,ESCA,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,GBM,HNSC,KICH,LIHC,LUAD,LUSC,Lung,MESO,PAAD,READ,SARC,SKCM,STAD,Small_Intestine_Terminal_Ileum,THCA,UCEC,UCS,Vagina
best10GTEx_TCGA,0.604,0.495,0.444,0.332,0.665,0.444,0.560,0.245,0.459,0.563,0.314,0.661,0.503,0.455,0.525,0.401,0.500,0.924,0.523,0.270,0.511,0.955,0.524,0.846,0.625,0.615,0.789,0.453,0.455,0.965,0.736,0.589,0.536,0.123,0.677,0.347,0.393,0.306,0.462,0.647,0.245
best10LFC_GTEx_TCGA,0.521,0.477,0.646,0.480,0.368,0.444,0.560,0.245,0.351,0.627,0.585,0.256,0.653,0.496,0.424,0.220,0.684,0.656,0.564,0.027,0.396,0.538,0.605,0.549,0.663,0.667,0.659,0.379,0.462,0.640,0.491,0.411,0.482,0.058,0.500,0.209,0.661,0.578,0.642,0.235,0.670
best10CuratedDegree1,0.896,0.818,0.792,0.807,0.534,0.778,0.440,0.396,0.574,0.754,0.280,0.752,0.776,0.309,0.522,0.808,0.627,0.938,0.584,0.396,0.644,0.982,0.683,0.912,0.747,0.923,0.906,0.485,0.595,1.000,0.868,0.430,0.679,0.097,0.839,0.271,0.688,0.827,0.783,0.794,0.617
best10CuratedDegree1_ce467ff,0.958,0.763,0.681,0.750,0.618,0.778,0.286,0.377,0.514,0.762,0.534,0.785,0.810,0.423,0.562,0.775,0.652,0.942,0.584,0.369,0.636,0.991,0.741,0.945,0.683,0.897,0.874,0.841,0.691,0.994,0.811,0.860,0.607,0.006,0.855,0.409,0.661,0.957,0.811,0.735,0.649
best10LFC_CuratedDegree1,0.729,0.692,0.840,0.639,0.527,0.889,0.516,0.396,0.595,0.627,0.703,0.397,0.694,0.398,0.576,0.654,0.620,0.902,0.572,0.342,0.618,0.991,0.641,0.890,0.708,0.846,0.740,0.595,0.645,0.986,0.792,0.561,0.679,0.019,0.758,0.307,0.723,0.711,0.736,0.794,0.681


In [9]:
selectBellowSenitivityRows = evalBest10LFCSensitivityDF.index == stage
display( evalBest10LFCSensitivityDF.loc[selectBellowSenitivityRows, belowSeries.values] )
aedwipDF = evalBest10LFCSensitivityDF.loc[selectBellowSenitivityRows, belowSeries.values]
aedwipDF.transpose().sort_values(by=stage)

id,ACC,Adipose_Visceral_Omentum,Artery_Coronary,BLCA,BRCA,Bladder,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Breast_Mammary_Tissue,CESC,COAD,Colon_Sigmoid,Colon_Transverse,ESCA,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,GBM,HNSC,KICH,LIHC,LUAD,LUSC,Lung,MESO,PAAD,READ,SARC,SKCM,STAD,Small_Intestine_Terminal_Ileum,THCA,UCEC,UCS,Vagina
best10LFC_GTEx_TCGA,0.521,0.477,0.646,0.48,0.368,0.444,0.56,0.245,0.351,0.627,0.585,0.256,0.653,0.496,0.424,0.22,0.684,0.656,0.564,0.027,0.396,0.538,0.605,0.549,0.663,0.667,0.659,0.379,0.462,0.64,0.491,0.411,0.482,0.058,0.5,0.209,0.661,0.578,0.642,0.235,0.67


,best10LFC_GTEx_TCGA
id,
ESCA,0.027
SARC,0.058
STAD,0.209
CESC,0.220
UCS,0.235
Brain_Anterior_cingulate_cortex_BA24,0.245
Brain_Hypothalamus,0.256
Brain_Caudate_basal_ganglia,0.351
BRCA,0.368


## find difference between best10CuratedDegree1_ce467ff and best10CuratedDegree1
These two runs have the highest mean sensitivity

<span style="color:red;background-color:yellow">manual curation </span>  
look for neg difference. These are categories where best10CuratedDegree1 did better. we should be able to 
 


In [10]:
# threshold = 0.7
metricCols = ['mean_sensitivity', 'std_sensitivity',
       'median_sensitivity', 'numGenes', 'numTypes', 'numDegree1',
       'numAboveThreshold']

categoryCols = ~ evalBest10LFCSensitivityDF.columns.isin(metricCols)

selectBestRuns = ["best10CuratedDegree1_ce467ff", "best10CuratedDegree1", ]
bestRunsDF = evalBest10LFCSensitivityDF.loc[selectBestRuns , categoryCols]
bestRunsDF.loc['diff'] = bestRunsDF.loc['best10CuratedDegree1_ce467ff', :] - bestRunsDF.loc['best10CuratedDegree1', :] #.transpose()
bestRunsDF

id,ACC,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,BLCA,BRCA,Bladder,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,CESC,CHOL,COAD,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Cervix_Endocervix,Colon_Sigmoid,Colon_Transverse,DLBC,ESCA,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,GBM,HNSC,Heart_Atrial_Appendage,Heart_Left_Ventricle,KICH,KIRC,KIRP,Kidney_Cortex,LGG,LIHC,LUAD,LUSC,Liver,Lung,MESO,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,OV,Ovary,PAAD,PCPG,PRAD,Pancreas,Pituitary,Prostate,READ,SARC,SKCM,STAD,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,TGCT,THCA,THYM,Testis,Thyroid,UCEC,UCS,UVM,Uterus,Vagina,Whole_Blood
best10CuratedDegree1_ce467ff,0.958,0.935,0.763,0.994,0.977,0.681,0.967,0.750,0.618,0.778,0.286,0.377,0.514,0.868,0.938,0.935,0.762,0.534,0.785,0.810,0.423,1.0,0.393,0.562,0.775,0.909,0.652,0.99,1.0,1.000,0.942,0.584,0.966,0.369,0.636,0.991,0.741,0.945,0.683,0.981,0.934,0.897,0.872,0.902,1.000,0.786,0.874,0.841,0.691,0.956,0.994,0.811,0.732,1.000,0.987,0.881,0.972,0.86,0.991,0.977,1.0,0.994,0.871,0.607,0.006,0.855,0.409,0.771,0.707,0.661,1.0,0.749,0.888,0.957,0.819,0.995,0.99,0.811,0.735,1.000,0.929,0.649,1.000
best10CuratedDegree1,0.896,0.945,0.818,1.000,0.985,0.792,0.935,0.807,0.534,0.778,0.440,0.396,0.574,0.922,0.903,0.882,0.754,0.280,0.752,0.776,0.309,1.0,0.369,0.522,0.808,0.909,0.627,1.00,1.0,0.833,0.938,0.584,1.000,0.396,0.644,0.982,0.683,0.912,0.747,0.860,0.961,0.923,0.862,0.873,0.981,0.770,0.906,0.485,0.595,0.875,1.000,0.868,0.804,0.992,0.987,0.812,0.972,0.43,0.880,0.903,1.0,0.822,0.864,0.679,0.097,0.839,0.271,0.813,0.760,0.688,1.0,0.744,0.775,0.827,0.917,1.000,0.99,0.783,0.794,0.958,0.882,0.617,0.987
diff,0.062,-0.010,-0.055,-0.006,-0.008,-0.111,0.032,-0.057,0.084,0.000,-0.154,-0.019,-0.060,-0.054,0.035,0.053,0.008,0.254,0.033,0.034,0.114,0.0,0.024,0.040,-0.033,0.000,0.025,-0.01,0.0,0.167,0.004,0.000,-0.034,-0.027,-0.008,0.009,0.058,0.033,-0.064,0.121,-0.027,-0.026,0.010,0.029,0.019,0.016,-0.032,0.356,0.096,0.081,-0.006,-0.057,-0.072,0.008,0.000,0.069,0.000,0.43,0.111,0.074,0.0,0.172,0.007,-0.072,-0.091,0.016,0.138,-0.042,-0.053,-0.027,0.0,0.005,0.113,0.130,-0.098,-0.005,0.00,0.028,-0.059,0.042,0.047,0.032,0.013


In [12]:
xxxRunsDF = evalBest10LFCSensitivityDF.loc[["best10CuratedDegree1_ce467ff", "best10LFC_CuratedDegree1", ] , categoryCols]
xxxRunsDF.loc['diff'] = xxxRunsDF.loc['best10CuratedDegree1_ce467ff', :] - xxxRunsDF.loc['best10LFC_CuratedDegree1', :] #.transpose()
xxxRunsDF

id,ACC,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,BLCA,BRCA,Bladder,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,Brain_Cerebellum,Brain_Cortex,Brain_Frontal_Cortex_BA9,Brain_Hippocampus,Brain_Hypothalamus,Brain_Nucleus_accumbens_basal_ganglia,Brain_Putamen_basal_ganglia,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Breast_Mammary_Tissue,CESC,CHOL,COAD,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Cervix_Endocervix,Colon_Sigmoid,Colon_Transverse,DLBC,ESCA,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,GBM,HNSC,Heart_Atrial_Appendage,Heart_Left_Ventricle,KICH,KIRC,KIRP,Kidney_Cortex,LGG,LIHC,LUAD,LUSC,Liver,Lung,MESO,Minor_Salivary_Gland,Muscle_Skeletal,Nerve_Tibial,OV,Ovary,PAAD,PCPG,PRAD,Pancreas,Pituitary,Prostate,READ,SARC,SKCM,STAD,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,TGCT,THCA,THYM,Testis,Thyroid,UCEC,UCS,UVM,Uterus,Vagina,Whole_Blood
best10CuratedDegree1_ce467ff,0.958,0.935,0.763,0.994,0.977,0.681,0.967,0.750,0.618,0.778,0.286,0.377,0.514,0.868,0.938,0.935,0.762,0.534,0.785,0.810,0.423,1.000,0.393,0.562,0.775,0.909,0.652,0.99,1.0,1.0,0.942,0.584,0.966,0.369,0.636,0.991,0.741,0.945,0.683,0.981,0.934,0.897,0.872,0.902,1.000,0.786,0.874,0.841,0.691,0.956,0.994,0.811,0.732,1.000,0.987,0.881,0.972,0.860,0.991,0.977,1.000,0.994,0.871,0.607,0.006,0.855,0.409,0.771,0.707,0.661,1.0,0.749,0.888,0.957,0.819,0.995,0.990,0.811,0.735,1.0,0.929,0.649,1.000
best10LFC_CuratedDegree1,0.729,0.957,0.692,0.955,0.977,0.840,0.952,0.639,0.527,0.889,0.516,0.396,0.595,0.899,0.861,0.810,0.627,0.703,0.397,0.694,0.398,0.947,0.690,0.576,0.654,0.955,0.620,1.00,1.0,1.0,0.902,0.572,1.000,0.342,0.618,0.991,0.641,0.890,0.708,0.829,0.819,0.846,0.716,0.717,0.981,0.718,0.740,0.595,0.645,0.993,0.986,0.792,0.753,0.998,0.989,0.625,0.963,0.561,0.815,0.812,0.995,0.793,0.898,0.679,0.019,0.758,0.307,0.325,0.940,0.723,1.0,0.726,0.800,0.711,0.861,0.995,0.982,0.736,0.794,1.0,0.941,0.681,0.976
diff,0.229,-0.022,0.071,0.039,0.000,-0.159,0.015,0.111,0.091,-0.111,-0.230,-0.019,-0.081,-0.031,0.077,0.125,0.135,-0.169,0.388,0.116,0.025,0.053,-0.297,-0.014,0.121,-0.046,0.032,-0.01,0.0,0.0,0.040,0.012,-0.034,0.027,0.018,0.000,0.100,0.055,-0.025,0.152,0.115,0.051,0.156,0.185,0.019,0.068,0.134,0.246,0.046,-0.037,0.008,0.019,-0.021,0.002,-0.002,0.256,0.009,0.299,0.176,0.165,0.005,0.201,-0.027,-0.072,-0.013,0.097,0.102,0.446,-0.233,-0.062,0.0,0.023,0.088,0.246,-0.042,0.000,0.008,0.075,-0.059,0.0,-0.012,-0.032,0.024


# <span style="color:red;background-color:yellow">Esophagus </span>

In [13]:
esophCategories = ['Esophagus_Gastroesophageal_Junction','Esophagus_Mucosa', 
                   'Esophagus_Muscularis', 'ESCA', 'HNSC' ]

print(f'specificity')
display( evalBest10LFCSpecificityDF.loc[:, esophCategories] )

print(f'\nsensitivity')
evalBest10LFCSensitivityDF.loc[:, esophCategories]

specificity


id,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,ESCA,HNSC
best10GTEx_TCGA,0.996,0.995,0.996,0.999,0.996
best10LFC_GTEx_TCGA,0.997,0.998,0.985,1.000,0.994
best10CuratedDegree1,0.994,0.997,0.996,0.998,0.995
best10CuratedDegree1_ce467ff,0.994,0.996,0.995,0.999,0.994
best10LFC_CuratedDegree1,0.993,0.992,0.994,0.999,0.995



sensitivity


id,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,ESCA,HNSC
best10GTEx_TCGA,0.511,0.955,0.524,0.270,0.625
best10LFC_GTEx_TCGA,0.396,0.538,0.605,0.027,0.663
best10CuratedDegree1,0.644,0.982,0.683,0.396,0.747
best10CuratedDegree1_ce467ff,0.636,0.991,0.741,0.369,0.683
best10LFC_CuratedDegree1,0.618,0.991,0.641,0.342,0.708


# <span style="color:red;background-color:yellow"> Uterus</span>

In [14]:
uterusCategories = ['Uterus', 'UCEC', 'UCS' ]

print(f'specificity')
display( evalBest10LFCSpecificityDF.loc[:, uterusCategories] )

print(f'\nsensitivity')
evalBest10LFCSensitivityDF.loc[:, uterusCategories]

specificity


id,Uterus,UCEC,UCS
best10GTEx_TCGA,0.997,0.996,0.995
best10LFC_GTEx_TCGA,0.998,0.999,1.000
best10CuratedDegree1,0.998,0.996,0.996
best10CuratedDegree1_ce467ff,1.000,0.998,0.995
best10LFC_CuratedDegree1,0.997,0.998,0.996



sensitivity


id,Uterus,UCEC,UCS
best10GTEx_TCGA,0.894,0.462,0.647
best10LFC_GTEx_TCGA,0.812,0.642,0.235
best10CuratedDegree1,0.882,0.783,0.794
best10CuratedDegree1_ce467ff,0.929,0.811,0.735
best10LFC_CuratedDegree1,0.941,0.736,0.794


# <span style="color:red;background-color:yellow">AEDWIP </span>
